In [1]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

In [2]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install pydub
!pip install --upgrade youtube_dl
!apt-get update && apt-get install ffmpeg -y

  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.1.26-py3-none-any.whl size=2928467 sha256=23643b2a243238091811ead7e689f91be5ea5dcdde8cb29257864cb7316760d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-711g3xux/wheels/2d/79/97/7209650ef73114e0fe0603480da012ad3afacb9cae6b8acd9a
Successfully built yt-dlp
  Attempting uninstall: yt-dlp
    Found existing installation: yt-dlp 2025.1.26
    Uninstalling yt-dlp-2025.1.26:
      Successfully uninstalled yt-dlp-2025.1.26
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/

In [6]:
%%writefile Modulacion.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import os
import youtube_dl
from scipy.io import wavfile
import subprocess
from pyngrok import ngrok
@st.cache_data
def download_audio(link, output_name="audio.mp3"):
    command = [
        "yt-dlp",
        "--extract-audio",
        "--audio-format", "mp3",
        "-o", output_name,
        link
    ]
    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if not os.path.exists(output_name):
            st.error("Error: No se descargó correctamente el audio.")
            return False
        return True
    except subprocess.CalledProcessError as e:
        st.error(f"Error al descargar el audio: {e.stderr.decode()}")
        return False

def convert_mp3_to_wav(input_file="audio.mp3", output_file="output.wav"):
    if not os.path.exists(input_file):
        st.error(f"Error: El archivo {input_file} no existe.")
        return False

    command = [
        "ffmpeg",
        "-y",
        "-i", input_file,
        output_file
    ]
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if os.path.exists(output_file):
            return True
        else:
            st.error("Error: La conversión a WAV falló.")
            return False
    except subprocess.CalledProcessError as e:
        st.error(f"Error en ffmpeg: {e.stderr.decode()}")
        return False

@st.cache_data(hash_funcs={str: lambda x: x})
def procesar_audio(link, start_time, end_time, Im):
    # Eliminar archivos de audio previos
    for file in ["audio.mp3", "output.wav"]:
        if os.path.exists(file):
            os.remove(file)

    # Descargar y convertir el audio
    if download_audio(link):
        if convert_mp3_to_wav("audio.mp3", "output.wav"):
            st.success("Audio descargado y convertido correctamente.")

st.sidebar.title("Estudiantes")
st.sidebar.write("Yilder Rafael Epiayu")
st.sidebar.write("Sebastian Perez")

# Frecuencia portadora y mensaje
Fc = 100
Fm = 10
Am = 1

ti = 0
tf = 5 / Fm
Fs = 5 * Fc

t = np.arange(ti, tf, 1 / Fs)
m = Am * np.cos(2 * np.pi * Fm * t)  # Mensaje

Im = 0.8
Ac = max(abs(m)) / Im  # Amplitud de la portadora
c = Ac * np.sin(2 * np.pi * Fc * t)  # Portadora

# Señal modulada en AM
y = (1 + m / Ac) * c

# Transformadas de Fourier
Xfc = np.fft.fft(c)
Xfm = np.fft.fft(m)
Xfy = np.fft.fft(y)
vfre = np.fft.fftfreq(len(c), 1 / Fs)

# Gráficas en Streamlit
st.title("Modulación AM")
st.markdown("""
El proceso de modulación, usado principalmente en telecomunicaciones, es un proceso usado en la transmisión de señales por un medio (ya sea físico o no).

Se tiene en este caso la señal que se quiere transmitir (señal mensaje). Estas señales son principalmente de baja frecuencia, por lo que para transmitirlas de un lado a otro, se requieren de equipos muy especializados y en tanto, costosos, por lo que se aplica esta técnica.

La señal mensaje se mezcla con una señal portadora (puede ser una señal senoidal), de muy alta frecuencia, que la va a transportar por el medio.

Al mezclarlas, la señal mensaje modula a partir de su amplitud a la señal de alta frecuencia, haciendo que el mensaje original quede inscrito allí y se pueda transmitir fácilmente debido a que sigue conservando la alta frecuencia de la señal portadora
""")
st.markdown("""
- Se requiere entonces de una portadora (carrier) senoidal/cosenoidal:
""")
st.latex("c(t)=A_c\sin(2\pi F_c t)")
st.markdown("""
- Para ilustrar el proceso, se trabaja con un mensaje tipo sen/cos:
""")
st.latex("m(t) = A_m \cos(2\pi F_m t)")
st.markdown("""
- La modulación AM puede calibrarse en términos del índice de modulación:
""")
st.latex("I_m = \\frac{\\text{Peak}\{|m(t)|\}}{A_c}")
st.markdown("""
donde :
""")
st.latex(r"\text{Peak}\{|m(t)|\} \text{es el valor pico (máximo) de}\,|m(t)|")
st.markdown("""
- La señal a transmitir, señal modulada, en AM se cálcula como:
""")
st.latex("y(t)= (1+\\frac{m(t)}{A_c})c(t)")
st.subheader("Señales en el tiempo")
# Gráfica de la portadora
st.markdown("""
Gráfica portadora y mensaje
""")
plt.figure()
plt.plot(t, c, label='Portadora')
plt.plot(t, m, label='Mensaje')
plt.xlabel('$t[s]$')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt.gcf())
# Señal modulada
st.subheader("""
Gráfica de la señal modulada
""")
st.markdown("""Se modula la señal a partir de la señal mensaje y señal portadora
""")
plt.figure()
plt.plot(t, y, label='Señal Modulada')
plt.xlabel('$t[s]$')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt.gcf())
st.subheader("A partir de la fft se calcula el espectro de la señal portadora, mensaje y modulada")
st.markdown("""Espectro de portadora""")

# Espectro de la portadora
plt.figure()
plt.plot(vfre, abs(Xfc), label='Portadora')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())
st.markdown("""Espectro del mensaje""")
# Espectro del mensaje
plt.figure()
plt.plot(vfre, abs(Xfm), label='Mensaje')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())
st.markdown("""Espectro de señal modulada""")
# Espectro de la señal modulada
plt.figure()
plt.plot(vfre, abs(Xfy), label='Modulado')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())


st.header("Modulación de Audio desde YouTube")

# Entrada de link de YouTube
link = st.text_input("Ingrese el link del video de YouTube")


if link:
    with st.spinner("Descargando audio..."):
        #audio_file = download_ytvid_as_mp3(link, "audio")
        #os.system(f"ffmpeg -y -i {audio_file} output.wav")
        #!yt-dlp --extract-audio -o "audio" --audio-format mp3 {link}
        download_audio(link)
        #os.system(f"ffmpeg -y -i {audio_file} output.wav")
        #!ffmpeg -y -i audio.mp3 output.wav
        convert_mp3_to_wav("audio.mp3", "output.wav")
        #os.remove(audio_file)

    # Cargar el audio
    audio_file = "output.wav"
    x, fs = sf.read(audio_file)

    # Seleccionar el fragmento a modular
    st.subheader("Selección de Fragmento de Audio")
    start_time = st.slider("Tiempo de inicio (segundos)", 0, int(len(x)/fs) - 5, 20)
    end_time = start_time + 5

    xs = x[int(start_time*fs):int(end_time*fs), 0]
    tt = np.arange(start_time, end_time, 1/fs)
    procesar_audio(link, start_time, end_time, Im)
    # Gráfica del fragmento de audio
    plt.figure()
    plt.plot(tt, xs, label="Canal izquierdo")
    plt.xlabel('$t[s]$')
    plt.ylabel('$x(t)$')
    plt.legend()
    st.pyplot(plt.gcf())
    formula_placeholder = st.empty()

    with formula_placeholder.container():
      st.markdown("""
        Si el pedazo de la cancion es la señal de mensaje y la señal portadora es:
    """)
      st.latex("c(t)=A_c\\sin(2\\pi F_c t)")
      st.latex(r"F_c=1000Hz")

    # Definir índice de modulación
      Im = st.slider("Seleccione el índice de modulación", min_value=0.1, max_value=1.0, value=0.8)

   # st.markdown("""
   # Si el pedazo de la cancion es la señal de mensaje y la señal portadora es:
   # """)
   # st.latex("c(t)=A_c\sin(2\pi F_c t)")
   # st.latex(r"F_c=1000Hz")
    # Definir índice de modulación
   # Im = st.slider("Seleccione el índice de modulación", min_value=0.1, max_value=1.0, value=0.8)

    # Frecuencia portadora (fijada en 1000 Hz)
    #Fc = 1000

    # Señal modulada en AM
    Ac = max(abs(xs)) / Im
    t = np.arange(start_time, end_time, 1/fs)
    c = Ac * np.sin(2 * np.pi * Fc * t)
    y = (1 + xs / Ac) * c

    # Espectro de Frecuencia
    Xfc = np.fft.rfft(c)
    Xfm = np.fft.rfft(xs)
    Xfy = np.fft.rfft(y)
    vfre = np.fft.rfftfreq(len(c), 1 / fs)
    plt.figure()
    plt.plot(t,c,label='portadora')
    plt.plot(t,xs,label='mensaje')
    plt.legend()
    st.pyplot(plt.gcf())
    st.subheader("Señal modulada en AM")
    plt.figure()
    plt.plot(t,y)
    plt.xlabel('$t[s]$')
    plt.ylabel('$y(t)$')
    st.pyplot(plt.gcf())

    st.subheader("Espectro de Frecuencia del Audio Modulado")
    st.markdown("""- A continuación, se presenta el espectro de la señal mensaje (cancion), portadora y modulada.""")
    # Espectro de la portadora
    plt.figure()
    plt.plot(vfre, abs(Xfc), label='Portadora')
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.legend()
    st.pyplot(plt.gcf())
    # Espectro del mensaje
    plt.figure(figsize=(8, 6))

    # Subplot 1: Espectro modulado
    plt.subplot(2, 1, 1)
    plt.plot(vfre, abs(Xfy), label='Modulado', color="r")
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.legend()
    plt.ylim([min(abs(Xfy)), 1.1*max(abs(Xfy))])

    # Subplot 2: Espectro del mensaje
    plt.subplot(2, 1, 2)
    plt.plot(vfre, abs(Xfm), label='Mensaje')
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|X(f)|$')
    plt.legend()
    plt.ylim([min(abs(Xfm)), 1.1*max(abs(Xfm))])

    st.pyplot(plt.gcf())  # Mostrar la figura con los subplots
    plt.close()


    # Reproducir el audio
    st.header("Reproducción de Audio")
    st.subheader("Audio de la portadora ")
    st.audio(c, format='audio/wav', sample_rate=fs, start_time=start_time)
    st.subheader("Audio Original")
    st.audio(xs, format='audio/wav', sample_rate=fs, start_time=0)
    st.subheader("Audio Modulado")
    st.audio(y, format='audio/wav', sample_rate=fs, start_time=0)



Overwriting Modulacion.py


In [7]:
#token = '2sVStkh8Ikr1VuPibhJLJ7MScHZ_2xoJoXpoATzLidEagosJj' colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok
token = '2sVZMGlAbCOQAZOYFs2BaQ6HPOn_FLS6WQVfT8De6w9kpPS3'

In [8]:
from pyngrok import ngrok
# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run Modulacion.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://1110-34-135-68-109.ngrok-free.app
